In [18]:
import polars as pl
from neo4j import GraphDatabase
from google import genai
from extract_instructions import check_if_there_are_facts
from dotenv import load_dotenv
import os
import time
import requests

dotenv_path = os.path.abspath("../.env")  # Adjust the path accordingly
load_dotenv(dotenv_path)

client = genai.Client(api_key=os.environ["GEMINI_API"])

API_KEY = os.environ["CL_API_KEY"]
# defining a params dict for the parameters to be sent to the API
header =  {"Authorization": f"Token {API_KEY}"}

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "fairusecases")

In [19]:
## Dealing with Conclusion
def get_all_opinions(tx):
    
    results = tx.run("""
        MATCH (f:Facts)-[]-(o:Opinion)-[]-(c:Case)
        RETURN c.WestLawCaseName AS Title, c.FiledDate AS DATE, c.ClusterID AS ClusterID, f.Document AS Document, o.Document AS Opinion
        """
    )
         
    return results.to_df()


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database="neo4j") as session:
            df = session.execute_read(get_all_opinions)

In [20]:
df = pl.from_pandas(df)

In [21]:
df = df.with_columns(
    pl.col("Document").map_elements(len).alias("DocLength"),
    pl.col("ClusterID").cast(pl.Utf8).map_elements(lambda x: x[:-2]),
    pl.col("Opinion").map_elements(len).alias("OpinionLength")
).filter(pl.col("DocLength") < 500)

In [27]:
## Dealing with Conclusion
def get_all_opinions(tx, title):
    
    results = tx.run("""
        MATCH (c:Case {WestLawCaseName: $title})-[]-(o:Opinion)-[:OF]-(p)
        DETACH DELETE p
        """, title = title
    )

    return results


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database="neo4j") as session:
            for title in df["Title"]:
                session.execute_write(get_all_opinions, title)

In [8]:
def get_opinions_from_cluster(cluster_id, header = header):

    URL = f"https://www.courtlistener.com/api/rest/v4/clusters/{cluster_id}"

        # sending get request and saving the response as response object
    r = requests.get(url = URL, headers = header)

    if r.status_code == 200:

        response = r.json()
        return response
    
    else:

        return { None }

In [ ]:
df = df.with_columns(
    pl.col("ClusterID").map_elements(get_opinions_from_cluster).alias("Opinions")
)

In [ ]:
t = df["Opinions"][23]

In [ ]:
t